#### 1. Import packages

In [7]:
import numpy as np
import copy
import math
import pandas as pd

#### 2. Problem Statement
This work uses multiple linear regression to determine the dependent variable crosspoint saturation (CSw) of water and CO2 relative permeability curves from the irreducible water saturation (Swi) and true reference cross point saturation (RCS). The first independent variable Swi is a decimal that has the range of 0–1. To constrain this range, another variable true reference cross point saturation (RCS) is also introduced. RCS has physical meaning and is half of the dynamic space:
RCS = 0.5 × (1 - Swi).
This project has limited data, PRB#1 data is for training and PRB#2 data is for testing.

#### 3. Dataset
The training data is from the PRB#1 core samples relative permeability curves. First variable x(j=1) is Swi, and second x(j=2) is RCS. y is CS. There are eight samples from Lakota, Hulett, and Minnelusa formations.

In [8]:
df = pd.read_csv('CS_project.csv')
train = df.iloc[0:8]
# take a look at the dataset
train

,Sample No.,Swi,true RCS,CSw
0,Lakota 8031.4,0.540999,0.229500,0.770701
1,Lakota 8035.4,0.580368,0.209816,0.787591
2,Hulett 8307.7,0.469720,0.265140,0.743720
3,Hulett 8325.8,0.579942,0.210029,0.817819
4,Hulett 8332.6,0.526892,0.236554,0.712748
5,Minnelusa 9366.8,0.406033,0.296984,0.652225
6,Minnelusa 9464.2,0.416358,0.291821,0.747614
7,Minnelusa 9529.3,0.491233,0.254384,0.725876


In [9]:
X_train = np.asanyarray(train[['Swi', 'true RCS']])
# asanyarray: Convert the input to an ndarray, but pass ndarray subclasses through.
X_train

array([[0.54099923, 0.22950038],
       [0.58036782, 0.20981609],
       [0.46972036, 0.26513982],
       [0.5799417 , 0.21002915],
       [0.52689179, 0.2365541 ],
       [0.4060325 , 0.29698375],
       [0.41635819, 0.2918209 ],
       [0.49123282, 0.25438359]])

In [10]:
y_train = np.asanyarray(train[['CSw']])
y_train

array([[0.770701 ],
       [0.787591 ],
       [0.74372  ],
       [0.817819 ],
       [0.712748 ],
       [0.652225 ],
       [0.7476135],
       [0.7258761]])

In [11]:
test = df.iloc[8:11]
test

,Sample No.,Swi,true RCS,CSw
8,Lakota 8063,0.591990,0.204005,0.773270
9,Hulett 8330,0.521616,0.239192,0.766865
10,Minnelusa C 9487,0.409992,0.295004,0.681677


In [12]:
X_test = np.asanyarray(test[['Swi', 'true RCS']])
X_test

array([[0.59198969, 0.20400515],
       [0.52161571, 0.23919214],
       [0.40999183, 0.29500408]])

In [13]:
y_test = np.asanyarray(test[['CSw']])
y_test

array([[0.77327 ],
       [0.766865],
       [0.681677]])

#### 4. Cost Function

In [15]:
def compute_cost(X, y, w, b):
    '''
    compute cost
    Args:
      X (ndarray(m, n)) : Variables, m samples with n features
      y (ndarray(m))    : m target values
      w (ndarray(n,))   : n features' parameters
      b (scalar)        : parameter

    Returns:
      cost (scalar)     : cost (2* average of squared error)
    '''
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        f_wb = np.dot(X[i], w) + b
        cost = cost + (f_wb - y[i])**2

    cost = cost/2/m
    return cost

#### 5. Compute gradient

In [17]:
def compute_gradient(X, y, w, b):
    '''
    compute gradient descent: define dj_dw and dj_db 
    Args:
      X (ndarray(m, n)) : Variables, m samples with n features
      y (ndarray(m))    : m target values
      w (ndarray(n,))   : n features' parameters
      b (scalar)        : parameter

    Returns:
      dj_dw, dj_db 
    '''
    # sample size and parameter size
    m = X.shape[0]
    n = X.shape[1]

    # define dj_dw and dj_db
    dj_dw = np.zeros(n)
    dj_db = 0.

    # gradient descent functions
    for i in range(m):
        diff = np.dot(X[i], w) + b - y[i]
        dj_db = dj_db + diff/m

        for j in range(n):
            # at certain i,j dj_dw[j] = :
            dj_dw[j] = dj_dw[j] + diff * X[i,j] / m
           
    return dj_dw, dj_db
            

In [18]:
# compute and display gradient
# test with X, y
# first_dj_dw, first_dj_db = compute_gradient(X, y, w_init, b_init)
# print('initial dj_dw is: ',first_dj_dw)
# print('initial dj_db is: ',first_dj_db)

#### 6. Gradient Descent with Multiple Variables

In [19]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_dw,dj_db = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))
        if i% math.ceil(num_iters / 10) == 0: 
            print("Iteration and cost are ",i, J_history[-1]) # [-1] last J_history
 
    return w, b, J_history #return final w,b and J history for graphing

In [20]:
# initialize parameters
n = X_train.shape[1]
initial_w = np.zeros(n)
initial_b = 0.

# some gradient descent settings
iterations = 10000
alpha = 5.0e-1

# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations)
print("b,w found by gradient descent: ",b_final,w_final)

m,_ = X_test.shape
for i in range(m):
    print("prediction: ",np.dot(X_test[i], w_final) + b_final, "target value: ", y_test[i])

<ipython-input-17-5402e573ce5e>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dj_dw[j] = dj_dw[j] + diff * X[i,j] / m


Iteration and cost are  0 [0.03339206]
Iteration and cost are  1000 [0.00047345]
Iteration and cost are  2000 [0.00047095]
Iteration and cost are  3000 [0.00047093]
Iteration and cost are  4000 [0.00047093]
Iteration and cost are  5000 [0.00047093]
Iteration and cost are  6000 [0.00047093]
Iteration and cost are  7000 [0.00047093]
Iteration and cost are  8000 [0.00047093]
Iteration and cost are  9000 [0.00047093]
b,w found by gradient descent:  [0.47851671] [ 0.54837035 -0.03492684]
prediction:  [0.79602105] target value:  [0.77327]
prediction:  [0.75620107] target value:  [0.766865]
prediction:  [0.69304051] target value:  [0.681677]
